In [9]:
pip install yahoo_fin

Note: you may need to restart the kernel to use updated packages.


In [1]:
import yahoo_fin.stock_info as yf
import pandas as pd

In [2]:
balance_sheet=[]
income_statement=[]
cfs=[]
years=[]
profitability_score=0
leverage_score=0
pe_ratio=0
payout_ratio=0
tax_rate=.2756
dol=0
dfl=0
current_ratio=0
debt_ratio=0
dol_dfl=pd.DataFrame(columns=['Ticker','dol','dfl'])
risk=pd.DataFrame(columns=['Ticker','dol','dfl','curr_ratio','debt_ratio'])

In [3]:
 tickers=["BRIGADE.NS","PRESTIGE.NS","GODREJPROP.NS","SUNTECK.NS","PHOENIXLTD.NS","OBEROIRLTY.NS","DLF.NS","SOBHA.NS","NBCC.NS","OMAXE.NS","MAHLIFE.NS"]


In [13]:
def get_data(ticker):
    global balance_sheet
    global income_statement
    global cfs
    global years
    balance_sheet=yf.get_balance_sheet(ticker)
    income_statement=yf.get_income_statement(ticker)
    cfs=yf.get_cash_flow(ticker)
    years=balance_sheet.columns
    

In [7]:
def profitability():
    global profitability_score
    #Scores 1 and 2 related to net income 
    net_income=income_statement[years[0]]['netIncome']
    net_income_py=income_statement[years[1]]['netIncome']
    ni_score =1 if net_income>0 else 0
    ni_score_2 =1 if net_income>net_income_py else 0 
    
    #Score 3- Operating Cash flow 
    op_cf=cfs[years[0]]['totalCashFromOperatingActivities']
    op_cf_score=1 if op_cf >0 else 0
    
    #Score 4 
    avg_assets=(balance_sheet[years[0]]['totalAssets']+balance_sheet[years[1]]['totalAssets'])/2
    avg_assets_py=(balance_sheet[years[1]]['totalAssets']+balance_sheet[years[2]]['totalAssets'])/2
    RoA=net_income/avg_assets
    print("Return On Assets->")
    print(RoA)
    RoA_py=net_income_py/avg_assets_py
    RoA_score=1 if RoA>RoA_py else 0
    #Score 5 Accruals
    total_assets=balance_sheet[years[0]]['totalAssets']
    accruals= op_cf/total_assets -RoA
    print("Accruals->")
    print(accruals)
    ac_score =1 if accruals>0 else 0
    
    profitability_score= ni_score +ni_score_2 + op_cf_score + RoA_score +ac_score 
#     print(profitability_score)
    

In [6]:
def leverage():
    global leverage_score
    global dol
    global dfl
    global debt_ratio
    global current_ratio
    #Score 6 - long-term debt ratio
    try:
        lt_debt=balance_sheet[years[0]]['longTermDebt']
        total_assets=balance_sheet[years[0]]['totalAssets']
        debt_ratio=lt_debt/total_assets
        debt_ratio_score=1 if debt_ratio<0.4 else 0
        print("debt Ratio->")
        print(debt_ratio)
    except:
        debt_ratio_score=1
    #Score 7
    current_assets=balance_sheet[years[0]]['totalCurrentAssets']
    current_liab=balance_sheet[years[0]]['totalCurrentLiabilities']
    current_ratio=current_assets/current_liab
    print("Current Ratio->")
    print(current_ratio)
    current_ratio_score=1 if current_ratio>1 else 0
    #Score 8 dol
    op_income=income_statement[years[0]]['operatingIncome']
    op_income_py=income_statement[years[1]]['operatingIncome']
    tot_Rev=income_statement[years[0]]['totalRevenue']
    tot_Rev_py=income_statement[years[1]]['totalRevenue']
    dol=(op_income-op_income_py)/(tot_Rev-tot_Rev_py)
    print("dol->")
    print(dol)
    #Score 9 dfl
    net_income=income_statement[years[0]]['netIncome']
    net_income_py=income_statement[years[1]]['netIncome']
    ebit=income_statement[years[0]]['ebit']
    ebit_py=income_statement[years[1]]['ebit']
    dfl=(net_income-net_income_py)/(ebit-ebit_py)
    print("dfl->")
    print(dfl)
    
    
    
    leverage_score=debt_ratio_score+current_ratio_score
#     print(leverage_score)
    
        
        
        
       

In [7]:
def operating_efficiency():
    #score 10 Gross Margin
    gp=income_statement[years[0]]['grossProfit']
    gp_py=income_statement[years[1]]['grossProfit']
    tot_rev=income_statement[years[0]]['totalRevenue']
    tot_rev_py=income_statement[years[1]]['totalRevenue']
    gm=(gp/tot_rev)
    print("Gross Margin->")
    print(gm)
    #Score 11 Asset Turnover
    avg_assets=(balance_sheet[years[0]]['totalAssets']+balance_sheet[years[1]]['totalAssets'])/2
    asset_turnover=(avg_assets/tot_rev)
    print("Asset-Turnover->")
    print(asset_turnover)
    

In [8]:
def pe(ticker):
    global pe_ratio
    pe_ratio=yf.get_quote_table(ticker)['PE Ratio (TTM)']
    if pe_ratio!=pe_ratio: #Check if NaN
        pe_ratio=0
    print("PE_RAtio->")
    print(pe_ratio)

In [9]:
def pay_out(ticker):
    global payout_ratio
    stat_data=yf.get_stats(ticker)
    payout_ratio=stat_data[stat_data['Attribute']=='Payout Ratio 4']['Value']
    
    print("Payout ratio->")
    print(payout_ratio)
      

In [41]:
for ticker in tickers:
     print("---------")
     print(ticker) 
     pay_out(ticker)

---------
BRIGADE.NS
Payout ratio->
24    12.41%
Name: Value, dtype: object
---------
PRESTIGE.NS
Payout ratio->
24    4.38%
Name: Value, dtype: object
---------
GODREJPROP.NS
Payout ratio->
24    0.00%
Name: Value, dtype: object
---------
SUNTECK.NS
Payout ratio->
24    61.73%
Name: Value, dtype: object
---------
PHOENIXLTD.NS
Payout ratio->
24    3.87%
Name: Value, dtype: object
---------
OBEROIRLTY.NS
Payout ratio->
24    7.67%
Name: Value, dtype: object
---------
DLF.NS
Payout ratio->
24    42.92%
Name: Value, dtype: object
---------
SOBHA.NS
Payout ratio->
24    42.49%
Name: Value, dtype: object
---------
NBCC.NS
Payout ratio->
24    43.10%
Name: Value, dtype: object
---------
OMAXE.NS
Payout ratio->
24    0.00%
Name: Value, dtype: object
---------
MAHLIFE.NS
Payout ratio->
24    13.93%
Name: Value, dtype: object


In [64]:
for ticker in tickers:
#     try:
        get_data(ticker)
        print("---------")
        print(ticker)
        print("---------")
        profitability()
        leverage()
        operating_efficiency()
        pe(ticker)
        new_row={'Ticker':ticker,
                  'dol':dol,
                   'dfl':dfl,
                'curr_ratio':current_ratio,
                 'debt_ratio':debt_ratio}
        risk=risk.append(new_row,ignore_index=True)
#     except:
#         print(ticker +": something went wrong with this ticker")
  


---------
BRIGADE.NS
---------
Return On Assets->
0.005712680535751013
Accruals->
0.06237747915364471
debt Ratio->
0.289298732314919
Current Ratio->
1.1233639019554074
dol->
0.13743194668243056
dfl->
0.8955876231441654
Gross Margin->
0.44753533103462073
Asset-Turnover->
4.83157150574567
PE_RAtio->
39.05
---------
PRESTIGE.NS
---------
Return On Assets->
0.041209406494960805
Accruals->
0.02908007488238324
debt Ratio->
0.13148360437654585
Current Ratio->
1.2325815735941505
dol->
0.34725480994838104
dfl->
5.514527027027027
Gross Margin->
0.47221222317865247
Asset-Turnover->
4.36751702011112
PE_RAtio->
14.56
---------
GODREJPROP.NS
---------
Return On Assets->
0.020698024533124065
Accruals->
-0.04606823139062897
debt Ratio->
0.05616854157258436
Current Ratio->
1.8832077905552242
dol->
0.43834673006528546
dfl->
1.1658093536792717
Gross Margin->
0.34578164043663145
Asset-Turnover->
9.329013414580684
PE_RAtio->
90.85
---------
SUNTECK.NS
---------
Return On Assets->
0.005254769033949198
Accru

In [12]:
print(balance_sheet)

endDate                         2022-03-31    2021-03-31    2020-03-31  \
Breakdown                                                                
intangibleAssets              4.680000e+05  3.730000e+05  1.791000e+06   
capitalSurplus                1.000079e+10  1.026087e+10  1.025183e+10   
totalLiab                     1.199825e+10  9.148242e+09  7.984826e+09   
totalStockholderEquity        1.788515e+10  1.631104e+10  1.701273e+10   
minorityInterest              4.910480e+08  4.197570e+08  4.193780e+08   
otherCurrentLiab              7.219283e+09  5.065007e+09  4.119199e+09   
totalAssets                   3.037444e+10  2.587904e+10  2.541693e+10   
commonStock                   1.545173e+09  5.138320e+08  5.136140e+08   
otherCurrentAssets            2.377241e+09  1.432606e+09  1.441619e+09   
retainedEarnings              4.885787e+09  3.338138e+09  4.070176e+09   
otherLiab                     6.096200e+07  2.225110e+08  1.334230e+08   
goodWill                      6.604470

In [12]:
# balance_sheet=yf.get_balance_sheet(ticker)
# years=balance_sheet.columns
# print(balance_sheet)

In [13]:
# print(balance_sheet.columns)
# print(balance_sheet[years[0]])

In [19]:
data=yf.get_quote_table("GODREJPROP.NS")
print(data)

{'1y Target Est': 1427.78, '52 Week Range': '1,129.55 - 2,074.90', 'Ask': '0.00 x 0', 'Avg. Volume': 630295.0, 'Beta (5Y Monthly)': 1.04, 'Bid': '0.00 x 0', "Day's Range": '1,308.00 - 1,337.00', 'EPS (TTM)': 14.63, 'Earnings Date': 'Feb 01, 2023 - Feb 06, 2023', 'Ex-Dividend Date': 'Jul 27, 2015', 'Forward Dividend & Yield': 'N/A (N/A)', 'Market Cap': '370.393B', 'Open': 1313.0, 'PE Ratio (TTM)': 91.06, 'Previous Close': 1308.05, 'Quote Price': 1332.25, 'Volume': 472930.0}


In [34]:
stat_data=yf.get_stats("GODREJPROP.NS")
stat_data[stat_data['Attribute']=='Payout Ratio 4']['Value']

24    0.00%
Name: Value, dtype: object

In [32]:
stat_data_2=yf.get_stats_valuation("GODREJPROP.NS")
print(type(stat_data_2))

<class 'pandas.core.frame.DataFrame'>


In [14]:
print(cfs)

endDate                                  2022-03-31    2021-03-31  \
Breakdown                                                           
investments                           -5.860000e+07  1.082170e+08   
changeToLiabilities                    2.553455e+09  1.066230e+09   
totalCashflowsFromInvestingActivities  1.232339e+09  1.104221e+09   
netBorrowings                          3.254830e+08  7.678000e+07   
totalCashFromFinancingActivities       1.215740e+08 -1.984420e+08   
issuanceOfStock                        2.479500e+07  2.810000e+05   
netIncome                              1.544903e+09 -7.173880e+08   
changeInCash                           8.339500e+08  2.255500e+08   
totalCashFromOperatingActivities      -5.199630e+08 -6.802290e+08   
depreciation                           6.465900e+07  6.845800e+07   
otherCashflowsFromInvestingActivities  6.043370e+08  9.485160e+08   
dividendsPaid                         -3.766000e+06 -4.354000e+06   
changeToInventory                 

In [15]:
print(income_statement)

endDate                            2022-03-31   2021-03-31   2020-03-31  \
Breakdown                                                                 
researchDevelopment                      None         None         None   
effectOfAccountingCharges                None         None         None   
incomeBeforeTax                     992805000   -777784000  -1962448000   
minorityInterest                    491048000    419757000    419378000   
netIncome                          1544903000   -717388000  -1934117000   
sellingGeneralAdministrative       1064997000    940660000   1168656000   
grossProfit                         806725000    399550000   1169699000   
ebit                               -814999000  -1004959000   -645252000   
operatingIncome                    -814999000  -1004959000   -645252000   
otherOperatingExpenses              491663000    393974000    569225000   
interestExpense                     -64662000    -93568000    -62958000   
extraordinaryItems       

In [26]:
def exp_growth_rate():
    capex=cfs[years[0]]['capitalExpenditures']
    depriciation=cfs[years[0]]['depreciation']
    curr_asset=balance_sheet[years[0]]['totalCurrentAssets']
    curr_liab=balance_sheet[years[0]]['totalCurrentLiabilities']
    cash=balance_sheet[years[0]]['cash']
    curr_asset_py=balance_sheet[years[1]]['totalCurrentAssets']
    curr_liab_py=balance_sheet[years[1]]['totalCurrentLiabilities']
    cash_py=balance_sheet[years[1]]['cash']
    change_in_wc=(curr_asset-curr_liab-cash)-(curr_asset_py-curr_liab_py-cash_py)
    ebit=income_statement[years[0]]['ebit']
    ebit_py=income_statement[years[1]]['ebit']
    capital_investment=cfs[years[0]]['investments']
    capital_investment_py=cfs[years[1]]['investments']
    re_investment_rate=(capex-depriciation +change_in_wc)/(ebit*(1-tax_rate))
    print("Reinvestment_rate->")
    print(re_investment_rate)
    
    return_on_capital=(ebit*(1-tax_rate))/capital_investment
    print("return_on_capital->")
    print(return_on_capital)
    return_on_capital_py=(ebit_py*(1-tax_rate))/capital_investment_py
    expected_growth_rate=(return_on_capital*re_investment_rate)+((return_on_capital-return_on_capital_py)/return_on_capital)
    print("expected_growth_rate->")
    print(expected_growth_rate)
    

In [28]:
tickers_1=["BRIGADE.NS","PRESTIGE.NS","GODREJPROP.NS","SUNTECK.NS","OBEROIRLTY.NS","DLF.NS","SOBHA.NS","NBCC.NS","OMAXE.NS","MAHLIFE.NS"]

In [29]:
for ticker in tickers_1:
        get_data(ticker)
        print("---------")
        print(ticker)
        print("---------")
        exp_growth_rate()
    

---------
BRIGADE.NS
---------
Reinvestment_rate->
0.17821020982867147
return_on_capital->
-0.3347186807300368
expected_growth_rate->
-0.686869640587544
---------
PRESTIGE.NS
---------
Reinvestment_rate->
-1.1367603504155952
return_on_capital->
12.65756487804878
expected_growth_rate->
-13.436796902725604
---------
GODREJPROP.NS
---------
Reinvestment_rate->
13.554951707503838
return_on_capital->
0.2367762103969809
expected_growth_rate->
3.808159175062235
---------
SUNTECK.NS
---------
Reinvestment_rate->
-3.2371401237791724
return_on_capital->
2.6088317646916535
expected_growth_rate->
-10.080709370568398
---------
OBEROIRLTY.NS
---------
Reinvestment_rate->
0.11312703463466128
return_on_capital->
-6.50341185158289
expected_growth_rate->
0.4047672326485088
---------
DLF.NS
---------
Reinvestment_rate->
0.048459564143151
return_on_capital->
22.338559520090318
expected_growth_rate->
2.2647111256244727
---------
SOBHA.NS
---------
Reinvestment_rate->
0.886050431882387
return_on_capital->
-

In [53]:
print(dol_dfl)
dol_dfl.to_csv("C:\\Bits_pilani\\3-1\\BAV\\DOL_DFL.csv")

           Ticker       dol        dfl
0      BRIGADE.NS  0.137432   0.895588
1     PRESTIGE.NS  0.347255   5.514527
2   GODREJPROP.NS  0.438347   1.165809
3      SUNTECK.NS  0.422394   0.395204
4   PHOENIXLTD.NS  0.573241   0.710695
5   OBEROIRLTY.NS  0.285124   1.683145
6          DLF.NS  0.940357   1.427775
7        SOBHA.NS  0.356493   0.246632
8         NBCC.NS  0.111064  -0.017053
9        OMAXE.NS  0.518438   1.026877
10     MAHLIFE.NS  0.083571  11.909302


In [66]:
print(risk)
risk.to_csv("C:\\Bits_pilani\\3-1\\BAV\\risk.csv")

           Ticker       dol        dfl  curr_ratio  debt_ratio
0      BRIGADE.NS  0.137432   0.895588    1.123364    0.289299
1     PRESTIGE.NS  0.347255   5.514527    1.232582    0.131484
2   GODREJPROP.NS  0.438347   1.165809    1.883208    0.056169
3      SUNTECK.NS  0.422394   0.395204    2.218327    0.079984
4   PHOENIXLTD.NS  0.573241   0.710695    1.870069    0.219097
5   OBEROIRLTY.NS  0.285124   1.683145    3.132590    0.137978
6          DLF.NS  0.940357   1.427775    2.315572    0.041701
7        SOBHA.NS  0.356493   0.246632    1.208048    0.063778
8         NBCC.NS  0.111064  -0.017053    1.047414    0.063778
9        OMAXE.NS  0.518438   1.026877    1.104861    0.043358
10     MAHLIFE.NS  0.083571  11.909302    1.889795    0.019797


In [16]:
data=yf.get_stats("GODREJPROP.NS")
print(data)

                           Attribute         Value
0                  Beta (5Y Monthly)          1.04
1                   52-Week Change 3       -36.24%
2            S&P500 52-Week Change 3       -19.75%
3                     52 Week High 3       2012.50
4                      52 Week Low 3       1129.55
5            50-Day Moving Average 3       1267.42
6           200-Day Moving Average 3       1358.15
7                Avg Vol (3 month) 3       606.38k
8                 Avg Vol (10 day) 3       502.05k
9               Shares Outstanding 5       278.02M
10      Implied Shares Outstanding 6           NaN
11                           Float 8        90.53M
12              % Held by Insiders 1        63.09%
13          % Held by Institutions 1        20.91%
14                    Shares Short 4           NaN
15                     Short Ratio 4           NaN
16                Short % of Float 4           NaN
17   Short % of Shares Outstanding 4           NaN
18     Shares Short (prior mont

In [30]:
data=yf.get_quote_table("GODREJPROP.NS")
print(data)

{'1y Target Est': 1427.78, '52 Week Range': '1,129.55 - 2,012.50', 'Ask': '0.00 x 0', 'Avg. Volume': 606377.0, 'Beta (5Y Monthly)': 1.04, 'Bid': '0.00 x 0', "Day's Range": '1,165.25 - 1,220.00', 'EPS (TTM)': 14.62, 'Earnings Date': 'Feb 01, 2023 - Feb 06, 2023', 'Ex-Dividend Date': 'Jul 27, 2015', 'Forward Dividend & Yield': 'N/A (N/A)', 'Market Cap': '326.146B', 'Open': 1218.0, 'PE Ratio (TTM)': 80.24, 'Previous Close': 1227.45, 'Quote Price': 1173.0999755859375, 'Volume': 723107.0}
